In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
import warnings
warnings.filterwarnings(action="ignore")

# Import Data

In [14]:
links_df= pd.read_csv("..\datasets\ml-latest-small\links.csv")
links_df

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [15]:
movies_df= pd.read_csv("..\datasets\ml-latest-small\movies.csv")
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [16]:
ratings_df= pd.read_csv("..\datasets\ml-latest-small\\ratings.csv")
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [17]:
tags_df= pd.read_csv("..\datasets\ml-latest-small\\tags.csv")
tags_df

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


## ItemBased Recommender (1)

In [ ]:
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [21]:
movies_df.query("movieId == 88448")

,movieId,title,genres
7656,88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama


In [18]:
def recommend_similar_movies(n,chosen_movie_id,criteria):
    cross_table = pd.pivot_table(data=ratings_df, values="rating",columns="movieId",index="userId")
    
    corr_table=pd.DataFrame({})
    corr_table["cross_corr"] = cross_table.corrwith(cross_table[chosen_movie_id]).dropna().drop(chosen_movie_id)
    #print(corr_table)
    
    corr_table_view = corr_table.merge(movies_df,on="movieId")[["title","genres","cross_corr","movieId"]]
    
    
    if criteria == "n_largest_corr":
        final_table =  corr_table_view.drop(columns="movieId").nlargest(n,"cross_corr")
        
    elif criteria == "50_plus_rate_count":
        rating = ratings_df.groupby("movieId").agg(rate_count = ("rating","count")).reset_index()
        final_table = (
            corr_table_view
            .merge(rating,on="movieId")
            .drop(columns="movieId")
            .query("rate_count >= 50")
            .nlargest(n,"cross_corr")
        )
        
    elif criteria == "weighted_rate":
        rating=(
        ratings_df
        .groupby("movieId")
        .agg(rate_mean =("rating","mean"),rate_count=("rating","count"))
        .reset_index())
        
        rating["weighted_rate"] = (rating.rate_count/rating.rate_count.sum()) *100* rating.rate_mean
        print(rating.weighted_rate.min(),rating.weighted_rate.max())
        final_table = (
            corr_table_view
            .merge(rating,on="movieId")
            .drop(columns="movieId")
            .query("weighted_rate >= 0.2")
            .nlargest(n,"cross_corr")
        )
        
    elif criteria == "bayes_average":
        # TODO: implement it 
        pass
        
    return final_table

In [19]:
recommend_similar_movies(5,148647,"50_plus_rate_count")

KeyError: 148647

In [88]:
recommend_similar_movies(5,1,"weighted_rate")

0.0004958546550835019 1.3923598714744734


,title,genres,cross_corr,rate_mean,rate_count,weighted_rate
47,Eye for an Eye (1996),Drama|Thriller,1.0,3.750000,6,0.022313
136,White Man's Burden (1995),Drama,1.0,3.166667,3,0.009421
233,Cobb (1994),Drama,1.0,3.333333,6,0.019834
237,Higher Learning (1995),Drama,1.0,2.666667,9,0.023801
258,Faster Pussycat! Kill! Kill! (1965),Action|Crime|Drama,1.0,3.700000,5,0.018347


In [69]:
movies_df[movies_df.movieId == 2]

,movieId,title,genres
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [42]:
cross_table = pd.pivot_table(data=ratings_df, values="rating",columns="movieId",index="userId")


c:\Python39\lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Python39\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,cross_corr
movieId,
1,-0.037776
2,0.577350
6,0.866025
10,-1.000000
11,-1.000000
...,...
183897,1.000000
184471,1.000000
187541,1.000000


In [45]:
cross_table.shape

(610, 9724)

In [44]:

corr_table=pd.DataFrame({})
corr_table["cross_corr"] = cross_table.corrwith(cross_table[168250])
#corr_table_view = corr_table.reset_index().merge(movies_df)[["title","genres","cross_corr","movieId"]]
corr_table

c:\Python39\lib\site-packages\numpy\lib\function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Python39\lib\site-packages\numpy\lib\function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)


,cross_corr
movieId,
1,-0.037776
2,0.577350
3,NaN
4,NaN
5,NaN
...,...
193581,NaN
193583,NaN
193585,NaN
